In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.naive_bayes import GaussianNB
#  importando los modulos confusion_matrix, roc_curve, auc, accuracy_score de sklearn.metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, classification_report
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_predict.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_train.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos Datos_Var_Rpta.xlsx
/kaggle/input/datatn-bancolombia-2019/DT19_IDs_predict.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos Datos_Trasaccionales.xlsx
/kaggle/input/datatn-bancolombia-2019/Trminos y Condiciones Datatn 2019.pdf
/kaggle/input/datatn-bancolombia-2019/presentacin Dataton BC 2019.pdf
/kaggle/input/datatn-bancolombia-2019/DT19_maestro_cdgtrn_cdgrpta.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train_lite.csv
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_train_lite.csv
/kaggle/input/datatn-bancolombia-2019/Metadatos maestro_cdgtrn_cdgrpta.xlsx
/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train.csv


In [141]:
dataY = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_Datos_Var_Rpta_train.csv')
dataX = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_train.csv')


In [142]:
dataY.head()

,id,f_analisis,var_rpta,segmento
0,1,201803,0,4
1,2,201604,0,0
2,3,201608,0,5
3,4,201706,0,4
4,5,201703,0,4


In [143]:
dataX.shape

(40533683, 8)

In [144]:
dataX.loc[dataX['id'] == 1, ['id', 'sesion']].tail(6)

,id,sesion
102632,1,2
102633,1,2
102634,1,2
102635,1,2
102636,1,1
102637,1,1


## Define function to Group by client id the amount of transactions on each chanel and merge the different parts of the dataset

In [168]:
def getFeatures(dataX): 
    grp = dataX.groupby(['id', 'canal'])['sesion'].nunique()
    trxcanal = grp.unstack(level=-1)
    trxcanal.columns = trxcanal.columns.values
    trxcanal = trxcanal.fillna(0)
    trxcanal['id'] = trxcanal.index.values
    del trxcanal.index.name
    return trxcanal

def mergeParts(dataA, dataB):
    data = pd.merge(dataA, dataB, left_on = 'id', right_on = 'id')
    #Convert segmento from categorical to dummy
    dummies = pd.get_dummies(data['segmento'], prefix = 'segmento')
    data = pd.concat([data.drop('segmento', axis = 1), dummies], axis = 1)
    return data

## Join the train info with the transactional info

In [169]:
data = mergeParts(dataY, getFeatures(dataX))
data.head()

,id,f_analisis,var_rpta,CANAL_1,CANAL_2,CANAL_3,CANAL_4,CANAL_5,CANAL_6,CANAL_7,CANAL_8,segmento_0,segmento_1,segmento_2,segmento_3,segmento_4,segmento_5
0,1,201803,0,0.0,0.0,0.0,0.0,69.0,0.0,0.0,0.0,0,0,0,0,1,0
1,2,201604,0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1,0,0,0,0,0
2,3,201608,0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1
3,4,201706,0,0.0,0.0,0.0,0.0,618.0,1.0,0.0,0.0,0,0,0,0,1,0
4,5,201703,0,0.0,1.0,0.0,0.0,70.0,46.0,0.0,2.0,0,0,0,0,1,0


## lets try a very simple model

In [147]:
colnames = data.columns.values.tolist()
predictors = colnames[3:17]
targetName = 'var_rpta'

## Very simple experimental setup

In [148]:
# Create a new array with the added features: features_two

features = data[predictors]
target = data[targetName]
# Split the data into train and test
trainX, testX, trainY, testY = train_test_split(features, target, test_size = 0.2, stratify = target )

print('Training: ')
print(trainX.shape, trainY.shape)
print(trainY.sum()/trainY.count())
print('Test: ')
print(testX.shape, testY.shape)
print(testY.sum()/testY.count())


Training: 
(61228, 14) (61228,)
0.07218919448618279
Test: 
(15308, 14) (15308,)
0.07218447870394565


## We need to undersample the dataset to avoid the unbalance of classes

In [149]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X_res, y_res = rus.fit_resample(trainX, trainY)

In [167]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# NN is sensitive to data scale. We must normilize
scaler = StandardScaler()  
trainXX = X_res
testXX = testX.copy()
# Don't cheat - fit only on training data
scaler.fit(trainX)  
trainXX = scaler.transform(trainXX)  
# apply same transformation to test data
testXX = scaler.transform(testXX)  

model3 = MLPClassifier(solver='adam', shuffle=True, learning_rate_init= 0.002, batch_size=200, alpha=1e-2, activation = 'tanh', max_iter=5000, tol = 1e-8, hidden_layer_sizes = [10, 8, 6, 4], learning_rate='adaptive', verbose = True)
model3.fit(trainXX, y_res)                     

#Print the score on the train data
print("On training")
print(model3.score(trainXX, y_res))
print(confusion_matrix(model3.predict(trainXX), y_res))

#Print the score on the test data
print("\nOn test")
print(model3.score(testXX, testY))
print(confusion_matrix(model3.predict(testXX), testY))

print("ROC: ")
print(roc_auc_score(testY, model3.predict(testXX)))
print(roc_auc_score(y_res, model3.predict(trainXX)))

Iteration 1, loss = 0.67864785
Iteration 2, loss = 0.64597875
Iteration 3, loss = 0.63385497
Iteration 4, loss = 0.63033339
Iteration 5, loss = 0.62678466
Iteration 6, loss = 0.62478363
Iteration 7, loss = 0.62382570
Iteration 8, loss = 0.62180395
Iteration 9, loss = 0.62063614
Iteration 10, loss = 0.61916873
Iteration 11, loss = 0.61933071
Iteration 12, loss = 0.61859746
Iteration 13, loss = 0.61777566
Iteration 14, loss = 0.61761245
Iteration 15, loss = 0.61688656
Iteration 16, loss = 0.61647033
Iteration 17, loss = 0.61604005
Iteration 18, loss = 0.61592082
Iteration 19, loss = 0.61549770
Iteration 20, loss = 0.61514128
Iteration 21, loss = 0.61514695
Iteration 22, loss = 0.61426536
Iteration 23, loss = 0.61521323
Iteration 24, loss = 0.61423210
Iteration 25, loss = 0.61423656
Iteration 26, loss = 0.61387887
Iteration 27, loss = 0.61419949
Iteration 28, loss = 0.61278195
Iteration 29, loss = 0.61401823
Iteration 30, loss = 0.61286828
Iteration 31, loss = 0.61244450
Iteration 32, los

In [86]:
trainXX

array([[ 0.29047319, -0.14282447, -0.18676381, ..., -0.50331468,
        -0.29827208, -0.29827208],
       [ 0.29047319, -0.14282447, -0.24366063, ...,  0.28727991,
         1.88549259,  1.88549259],
       [-1.77258039, -0.14282447,  0.21151393, ..., -0.32419559,
         0.22583144,  0.22583144],
       ...,
       [-1.77258039, -0.14282447, -0.01607335, ..., -0.42919644,
        -0.29827208, -0.29827208],
       [-1.25681699, -0.14282447, -0.35745427, ..., -0.50331468,
        -0.29827208, -0.29827208],
       [ 0.80623659, -0.14282447,  0.09772029, ..., -0.50331468,
        -0.29827208, -0.29827208]])

## Make a new prediction and save it in the output file

In [ ]:
data2Y = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_IDs_predict.csv')
data2X = pd.read_csv('/kaggle/input/datatn-bancolombia-2019/DT19_Datos_transaccionales_predict.csv')

data2 = mergeParts(data2Y, getFeatures(data2X))
features = data2[predictors]

test2XX = scaler.transform(features)  
#Make a prediction
data2Y['probabilidad'] =  model3.predict(test2XX)

data_final = data2Y[['id', 'probabilidad']]

# Save to a file. Find it in /kaggle/output/working/
data_final.to_csv("DT19_IDs_predict.csv", index=False)
